In [1]:
#importing libraries
from tensorflow import keras
 
from keras.preprocessing import text
from keras.utils import to_categorical  # Correct import

from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
# Define your generate_context_word_pairs function
def generate_context_word_pairs(corpus, window_size=2, vocab_size=None):
    context_length = window_size * 2

    for words in corpus:
        sentence_length = len(words)
       
        for index, word in enumerate(words):
            context_words = []
            label_word = []
            start = index - window_size
            end = index + window_size + 1

            context_words.append([words[i] for i in range(start, end) if 0 <= i < sentence_length and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, num_classes=vocab_size)  # Use to_categorical

            yield (x, y)

# Usage example
i = 0
for x, y in generate_context_word_pairs(wids, window_size=window_size, vocab_size=vocab_size):
    # Your code to process x and y goes here
    # You can remove the "if not in x[0]:" line as it's not necessary
    # Print statements for context and target are commented out
    # Adjust your processing logic here
    pass

    i += 1
    if i == 10:
        break


In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()


Epoch: 1 	Loss: 433.5798420906067

Epoch: 2 	Loss: 429.32352900505066

Epoch: 3 	Loss: 426.22231578826904

Epoch: 4 	Loss: 423.20136857032776

Epoch: 5 	Loss: 420.7082929611206



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.009698,-0.027849,0.032922,-0.042073,0.057088,0.010997,0.036490,0.021997,0.001519,-0.020211,...,-0.032118,0.031443,-0.051236,-0.030324,0.042352,0.022034,0.050413,-0.038029,-0.041890,0.017192
networks,0.002469,-0.064353,-0.006211,0.058111,0.003775,-0.038733,-0.033303,-0.024190,0.059570,0.060670,...,0.032534,0.055773,-0.031962,0.035971,0.000983,-0.060988,-0.023376,-0.020964,0.012990,0.003148
neural,0.041892,-0.043560,0.016013,0.039851,0.005543,0.043014,0.019749,0.036206,-0.022977,0.043284,...,-0.035868,-0.047640,0.034520,0.040007,0.023592,0.043557,-0.030098,-0.012024,0.029206,0.007278
and,0.039648,-0.046574,-0.027498,-0.045692,-0.046321,0.018051,-0.002250,-0.025804,0.047950,0.030197,...,0.044042,-0.032497,0.017453,-0.041595,-0.019190,-0.033120,0.001812,0.012192,0.031359,0.000539
as,0.005487,0.004296,0.028438,-0.024228,0.003662,-0.018548,0.036648,0.027335,-0.026443,0.035379,...,-0.035672,0.004895,0.030033,-0.025529,0.029138,0.018348,0.030057,-0.034538,0.024941,0.037755


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['results', 'programs', 'or', 'material', 'produced']}